In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'current_date={current_date}')

current_date=2023-05-24 11:00:00


In [3]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/47255
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2023-04-26 11:00:00 to 2023-05-24 10:00:00


2023-05-24 08:51:16,631 INFO: USE `taxi_demand_ytjqdg_featurestore`
2023-05-24 08:51:17,286 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_ytjqdg_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2023-04-25 11:00:00.000' AND `fg0`.`pickup_hour` <= TIMESTAMP '2023-05-25 10:00:00.000'


In [4]:
features.head()

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id
165,1.0,0.0,0.0,2.0,8.0,1.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,2023-05-24 11:00:00,1
250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-24 11:00:00,2
53,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-24 11:00:00,3
69,2.0,1.0,3.0,8.0,2.0,3.0,5.0,8.0,5.0,6.0,...,1.0,1.0,0.0,1.0,9.0,7.0,6.0,4.0,2023-05-24 11:00:00,4
67,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2023-05-24 11:00:00,5


In [5]:
from src.inference import load_model_from_registry, get_model_predictions

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/47255
Connected. Call `.close()` to terminate connection gracefully.

In [6]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,0.0,2023-05-24 11:00:00
1,2,0.0,2023-05-24 11:00:00
2,3,0.0,2023-05-24 11:00:00
3,4,3.0,2023-05-24 11:00:00
4,5,0.0,2023-05-24 11:00:00
...,...,...,...
260,261,32.0,2023-05-24 11:00:00
261,262,120.0,2023-05-24 11:00:00
262,263,125.0,2023-05-24 11:00:00
263,264,77.0,2023-05-24 11:00:00


### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [7]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/47255
Connected. Call `.close()` to terminate connection gracefully.


In [8]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

%3|1684929219.003|FAIL|Joaos-MacBook-Pro-2.local#producer-1| [thrd:ssl://c.app.hopsworks.ai:9092/bootstrap]: ssl://c.app.hopsworks.ai:9092/bootstrap: SSL handshake failed: error:1416F086:SSL routines:tls_process_server_certificate:certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (brew install openssl) (after 99ms in state SSL_HANDSHAKE)


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

%3|1684929220.027|FAIL|Joaos-MacBook-Pro-2.local#producer-1| [thrd:ssl://3.142.251.253:9092/bootstrap]: ssl://3.142.251.253:9092/bootstrap: SSL handshake failed: error:1416F086:SSL routines:tls_process_server_certificate:certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (brew install openssl) (after 108ms in state SSL_HANDSHAKE)
%3|1684929221.008|FAIL|Joaos-MacBook-Pro-2.local#producer-1| [thrd:ssl://3.145.101.110:9092/bootstrap]: ssl://3.145.101.110:9092/bootstrap: SSL handshake failed: error:1416F086:SSL routines:tls_process_server_certificate:certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (brew install openssl) (after 97ms in state SSL_HANDSHAKE)
%3|1684929222.034|FAIL|Joaos-MacBook-Pro-2.local#producer-1| [thrd:ssl://3.145.101.110:9092/bootstrap]: ssl://3.145.101.1

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/47255/jobs/named/model_predictions_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x17f33cc70>, None)